In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam

# 데이터 로드
df = pd.read_csv("/content/CarDataSets.csv", sep=',')

# NaN 값 확인 및 제거 (안정성을 위해 추가)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

X = df.drop(columns=['MPG_Highway'])
y = df[['MPG_Highway']]

# X와 y에 대해 각각 스케일러를 적용
scaler_X = MinMaxScaler()

# 스케일러 미적용시 값의 크기로 인해 NaN 오류 발생
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, shuffle=True)

# Test1
# model = keras.models.Sequential()
# model.add(keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(keras.layers.Dense(8, activation='relu'))
# model.add(keras.layers.Dense(1))

# Test2
# model = keras.models.Sequential()
# model.add(keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(keras.layers.Dense(16, activation='relu'))
# model.add(keras.layers.Dense(1))

# Test3
model = keras.models.Sequential()
model.add(keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1))

model.compile(loss='mse',
              optimizer='adam', # 또는 위에서 정의한 optimizer
              metrics=['mse'])

# 20회 이상 결과가 향상되지 않으면 자동으로 중단되게끔 합니다.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 모델의 이름을 정합니다.
# modelpath='/content/Test1.keras'
# modelpath='/content/Test2.keras'
modelpath='/content/Test3.keras'

# 최적화 모델을 업데이트하고 저장합니다.
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)

# 실행 관련 설정을 하는 부분입니다. 전체의 20%를 검증셋으로 설정합니다.
history = model.fit(X_train, y_train, validation_split=0.25, epochs=2000, batch_size=32, callbacks=[early_stopping_callback, checkpointer])

# 테스트 결과를 출력합니다.
score=model.evaluate(X_test, y_test)
print('Test MSE:', score[1])

Epoch 1/2000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 0.0845 - mse: 0.0845
Epoch 1: val_loss improved from inf to 0.03814, saving model to /content/Test3.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0657 - mse: 0.0657 - val_loss: 0.0381 - val_mse: 0.0381
Epoch 2/2000
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0466 - mse: 0.0466
Epoch 2: val_loss improved from 0.03814 to 0.02349, saving model to /content/Test3.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0355 - mse: 0.0355 - val_loss: 0.0235 - val_mse: 0.0235
Epoch 3/2000
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0158 - mse: 0.0158
Epoch 3: val_loss did not improve from 0.02349
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0169 - mse: 0.0169 - val_loss: 0.0240 - val_mse: 0.0240
Epoch 4/2000
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0154 - mse: 0.0154
Epoch 4: val_loss improved from 0.02349 to 0.02291, saving model to /content/Test3.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0176 - mse: 0.0176 - val_

In [ ]:
score=model.evaluate(X_test, y_test)
print('Test accuracy:', score)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0023 - mse: 0.0023
Test accuracy: [0.0015902240993455052, 0.0015902240993455052]


In [ ]:
idx=X_test.shape[0]
pred=model.predict(X_test)

for i in range(idx):
   print(f"예측: {pred[i]}, 실제: {y_test[i]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
예측: [0.11408751], 실제: [0.09259259]
예측: [0.33299902], 실제: [0.35185185]
예측: [0.33064568], 실제: [0.31481481]
예측: [0.3113932], 실제: [0.35185185]
예측: [0.27880797], 실제: [0.24074074]
예측: [0.24169795], 실제: [0.25925926]
예측: [0.26174214], 실제: [0.25925926]
예측: [0.14698313], 실제: [0.18518519]
예측: [0.2373621], 실제: [0.24074074]
예측: [0.32881054], 실제: [0.33333333]
예측: [0.24866448], 실제: [0.25925926]
예측: [0.24645674], 실제: [0.24074074]
예측: [0.28571048], 실제: [0.25925926]
예측: [0.25136316], 실제: [0.31481481]
예측: [0.6897416], 실제: [1.]
예측: [0.27402443], 실제: [0.31481481]
예측: [0.24851337], 실제: [0.14814815]
예측: [0.21290061], 실제: [0.14814815]
예측: [0.25383362], 실제: [0.25925926]
예측: [0.29668942], 실제: [0.2962963]
예측: [0.10335682], 실제: [0.09259259]
예측: [0.2623777], 실제: [0.27777778]
예측: [0.29200414], 실제: [0.33333333]
예측: [0.18374655], 실제: [0.2037037]
예측: [0.36607927], 실제: [0.38888889]
예측: [0.3622276], 실제: [0.40740741]
예측: [0.17500341], 실제: [0.16666667]
예측: [0.30757573], 실제: [0.3333333